Найдите количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».

In [1]:
query = '''
SELECT COUNT(*)
FROM stackoverflow.posts p

JOIN stackoverflow.post_types pt ON p.post_type_id=pt.id

WHERE (score > 300 OR favorites_count >= 100) AND (pt.type = 'Question')

'''


Сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлите до целого числа.

In [2]:
query = '''
WITH func AS (

SELECT EXTRACT(DAY FROM creation_date) AS day , COUNT(*)
FROM stackoverflow.post_types pt

JOIN stackoverflow.posts p ON pt.id=p.post_type_id

WHERE (EXTRACT(DAY FROM creation_date) BETWEEN 1 AND 18 AND EXTRACT(YEAR FROM creation_date) = 2008) AND type = 'Question' AND EXTRACT(MONTH FROM creation_date) = 11

GROUP BY day
)
SELECT ROUND(AVG(count),0)
FROM func
'''

Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей.

In [3]:
query = '''
SELECT COUNT(DISTINCT u.id)
FROM stackoverflow.users u 

JOIN stackoverflow.badges b ON u.id = b.user_id

WHERE u.creation_date::date = b.creation_date::date
'''

Сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос?

In [4]:
query = '''
SELECT COUNT(DISTINCT p.id) AS unique_posts
FROM stackoverflow.users u
JOIN stackoverflow.posts p ON u.id = p.user_id
JOIN stackoverflow.votes v ON p.id = v.post_id
WHERE u.display_name = 'Joel Coehoorn';
'''

Выгрузите все поля таблицы vote_types. Добавьте к таблице поле rank, в которое войдут номера записей в обратном порядке. Таблица должна быть отсортирована по полю id.

In [5]:
query = '''
SELECT *, ROW_NUMBER() OVER (ORDER BY id DESC) 
FROM stackoverflow.vote_types
ORDER BY id
'''

Отберите 10 пользователей, которые поставили больше всего голосов типа Close. Отобразите таблицу из двух полей: идентификатором пользователя и количеством голосов. Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.

In [6]:
query = '''
SELECT DISTINCT u.id AS unique_persons, COUNT(v.vote_type_id) AS count_golos

FROM stackoverflow.users u
JOIN stackoverflow.votes v ON u.id = v.user_id 
JOIN stackoverflow.vote_types vt ON v.vote_type_id = vt.id

WHERE name = 'Close'

GROUP BY unique_persons

ORDER BY count_golos DESC, unique_persons DESC 
LIMIT 10
'''

Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно.
Отобразите несколько полей:
идентификатор пользователя;
число значков;
место в рейтинге — чем больше значков, тем выше рейтинг.
Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.
Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.

In [7]:
query = '''
SELECT DISTINCT u.id AS unique_user, COUNT(b.id) AS count_badges, 
DENSE_RANK() OVER (ORDER BY COUNT(b.id) DESC) AS rank

FROM stackoverflow.users u
JOIN stackoverflow.badges b ON u.id = b.user_id
WHERE b.creation_date::date BETWEEN '2008-11-15' AND '2008-12-15'

GROUP BY unique_user
ORDER BY count_badges DESC, unique_user ASC
LIMIT 10
'''

Сколько в среднем очков получает пост каждого пользователя?
Сформируйте таблицу из следующих полей:
заголовок поста;
идентификатор пользователя;
число очков поста;
среднее число очков пользователя за пост, округлённое до целого числа.
Не учитывайте посты без заголовка, а также те, что набрали ноль очков.

In [8]:
query = '''
SELECT p.title AS title_posts, 
       u.id AS user_ind,  
       p.score AS score_posts,
       ROUND(AVG(p.score) OVER (PARTITION BY u.id)) AS avg_score_per_user

FROM stackoverflow.users u
JOIN stackoverflow.posts p ON u.id = p.user_id

WHERE p.score != 0 AND p.title  IS NOT NULL

'''

Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список.

In [9]:
query = '''
WITH func AS (
    SELECT u.id AS user_unique
    FROM stackoverflow.users u 
    JOIN stackoverflow.badges b ON u.id = b.user_id
    GROUP BY u.id
    HAVING COUNT(b.id) > 1000
)

SELECT p.title
FROM stackoverflow.posts p
JOIN func f ON p.user_id = f.user_unique
WHERE p.title IS NOT NULL;

'''

Напишите запрос, который выгрузит данные о пользователях из Канады (англ. Canada). Разделите пользователей на три группы в зависимости от количества просмотров их профилей:
пользователям с числом просмотров больше либо равным 350 присвойте группу 1;
пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;
пользователям с числом просмотров меньше 100 — группу 3.
Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с количеством просмотров меньше либо равным нулю не должны войти в итоговую таблицу.

In [10]:
query = '''
SELECT          u.id AS unique_user, 
                u.views AS views_users,
                (CASE 
                    WHEN u.views >= 350 THEN 1
                    WHEN u.views < 350 AND u.views >= 100 THEN 2
                    WHEN u.views < 100 THEN 3
                END) AS groupp_name

FROM stackoverflow.users u 
WHERE (u.location LIKE '%Canada%') AND (u.views >= 1)

'''


Дополните предыдущий запрос. Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. Выведите поля с идентификатором пользователя, группой и количеством просмотров. Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.

In [11]:
query = '''
WITH grp AS (SELECT g.id,
                    g.views,
                    g.group,
                    MAX(g.views) OVER (PARTITION BY g.group) AS max     
             FROM (SELECT id,
                          views,
                          CASE
                             WHEN views >= 350 THEN 1
                             WHEN views < 100 THEN 3
                             ELSE 2
                          END AS group
                   FROM stackoverflow.users
                   WHERE location LIKE '%Canada%' AND views > 0) as g
              )
  
SELECT grp.id, 
       grp.views,  
       grp.group
FROM grp
WHERE grp.views = grp.max
ORDER BY grp.views DESC, grp.id;
'''

Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года. Сформируйте таблицу с полями:
номер дня;
число пользователей, зарегистрированных в этот день;
сумму пользователей с накоплением.

In [12]:
query = '''
WITH func AS (
SELECT EXTRACT(DAY FROM creation_date) AS numder_of_day, 
       COUNT(id) AS count_reg

FROM stackoverflow.users

WHERE creation_date::date BETWEEN '2008-11-01' AND '2008-11-30'

GROUP BY numder_of_day)

SELECT numder_of_day,count_reg,
       SUM(count_reg) OVER (ORDER BY numder_of_day)
FROM func
'''

Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста. Отобразите:
идентификатор пользователя;
разницу во времени между регистрацией и первым постом.

In [13]:
query = '''
WITH ranked_posts AS (
    SELECT u.id AS user_id, 
           u.creation_date AS registration_date,
           p.creation_date AS post_creation_date,
           ROW_NUMBER() OVER (PARTITION BY u.id ORDER BY p.creation_date) AS post_rank
    FROM stackoverflow.users u
    JOIN stackoverflow.posts p ON u.id = p.user_id
)
SELECT user_id,
       (post_creation_date - registration_date) AS time_difference
FROM ranked_posts
WHERE post_rank = 1
ORDER BY user_id;

'''

Выведите общую сумму просмотров у постов, опубликованных в каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить. Результат отсортируйте по убыванию общего количества просмотров.

In [14]:
query = '''
SELECT DATE_TRUNC('month', creation_date)::date AS data_month, SUM(views_count) AS count_views
       
FROM stackoverflow.posts
WHERE EXTRACT(YEAR FROM creation_date) = 2008
GROUP BY data_month
ORDER BY count_views DESC
'''

Выведите имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. Вопросы, которые задавали пользователи, не учитывайте. Для каждого имени пользователя выведите количество уникальных значений user_id. Отсортируйте результат по полю с именами в лексикографическом порядке.

In [15]:
query = '''
select u.display_name,
count(distinct p.user_id) 
from stackoverflow.posts p 
join stackoverflow.users u
on p.user_id=u.id
where p.post_type_id = 2 
and DATE_TRUNC('day', p.creation_date) >= DATE_TRUNC('day', u.creation_date) AND DATE_TRUNC('day', p.creation_date) <= DATE_TRUNC('day', u.creation_date) + INTERVAL '1 month'
group by u.display_name
having count(p.id)> 100
order by u.display_name


'''

Выведите количество постов за 2008 год по месяцам. Отберите посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Отсортируйте таблицу по значению месяца по убыванию.

In [16]:
query = '''
SELECT DATE_TRUNC('month', p.creation_date)::date,
       COUNT(p.id)
FROM stackoverflow.posts p
WHERE p.user_id IN
(SELECT  DISTINCT u.id
FROM stackoverflow.users u
JOIN stackoverflow.posts p ON p.user_id=u.id
WHERE DATE_TRUNC('month', u.creation_date)::date = '2008-09-01'
AND DATE_TRUNC('month', p.creation_date)::date = '2008-12-01')
GROUP BY DATE_TRUNC('month', p.creation_date)::date
ORDER BY 1 DESC
'''

Используя данные о постах, выведите несколько полей:
идентификатор пользователя, который написал пост;
дата создания поста;
количество просмотров у текущего поста;
сумма просмотров постов автора с накоплением.
Данные в таблице должны быть отсортированы по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.

In [17]:
query = '''
SELECT p.user_id,
       p.creation_date,
       p.views_count,
       SUM(views_count) OVER(PARTITION BY p.user_id ORDER BY creation_date) AS cumulative_count
FROM stackoverflow.posts p
ORDER BY p.user_id, p.creation_date
'''

Сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой? Для каждого пользователя отберите дни, в которые он или она опубликовали хотя бы один пост. Нужно получить одно целое число — не забудьте округлить результат.

In [18]:
query = '''
WITH func AS (
SELECT p.user_id,
       COUNT(DISTINCT DATE_TRUNC('day', p.creation_date)::date)

FROM stackoverflow.posts AS p
WHERE DATE_TRUNC('day', p.creation_date)::date BETWEEN '2008-12-01' AND '2008-12-07'
GROUP BY p.user_id)

SELECT CEILING(AVG(count))::int
FROM func
'''

На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? Отобразите таблицу со следующими полями:
Номер месяца.
Количество постов за месяц.
Процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.
Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлите значение процента до двух знаков после запятой.
Напомним, что при делении одного целого числа на другое в PostgreSQL в результате получится целое число, округлённое до ближайшего целого вниз. Чтобы этого избежать, переведите делимое в тип numeric.

In [19]:
query = '''
WITH func AS (
SELECT EXTRACT(MONTH FROM creation_date) AS month_post,
       COUNT(p.id) AS post_count
     
FROM stackoverflow.posts p
WHERE EXTRACT(MONTH FROM creation_date) BETWEEN 9 AND 12
GROUP BY month_post
)

SELECT *, ROUND((post_count::numeric/LAG(post_count) OVER()-1)*100,2)
FROM func
'''

Найдите пользователя, который опубликовал больше всего постов за всё время с момента регистрации. Выведите данные его активности за октябрь 2008 года в таком виде:
номер недели;
дата и время последнего поста, опубликованного на этой неделе.

In [20]:
query = '''
WITH us AS
(SELECT user_id,
       COUNT(id)
FROM stackoverflow.posts
GROUP BY 1
ORDER BY 2 DESC
LIMIT 1)
SELECT EXTRACT(WEEK FROM p.creation_date)::int week_creation,
       MAX(p.creation_date) creation_date
FROM us u 
JOIN stackoverflow.posts p ON u.user_id=p.user_id
WHERE DATE_TRUNC('month', p.creation_date)::date = '2008-10-01'
GROUP BY 1
'''